In [8]:
print('ok')

ok


In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from dotenv import load_dotenv

d:\OpenAI_Projects\End_to_End_Medical_Chatbot_Using_LLama2\venv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import os
load_dotenv()

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

In [18]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [19]:
extracted_data = load_pdf("D:/OpenAI_Projects/End_to_End_Medical_Chatbot_Using_LLama2/data/")

In [20]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [21]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [22]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [23]:
embeddings = download_hugging_face_embeddings()

In [24]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [41]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

In [42]:
index_name="medical-chatbot"

In [43]:
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    ) 

In [44]:
import time
from langchain_pinecone import PineconeVectorStore


In [45]:
namespace = "wondervector5000"

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
    namespace=namespace 
)

time.sleep(1)

In [46]:
index = pc.Index(index_name)

for ids in index.list(namespace=namespace):
    query = index.query(
        id=ids[0], 
        namespace=namespace, 
        top_k=1,
        include_values=True,
        include_metadata=True
    )
    print(query)

{'matches': [{'id': '000f7eb7-3ea0-4fac-b6a1-c969b7e629e9',
              'metadata': {'page': 64.0,
                           'source': 'D:\\OpenAI_Projects\\End_to_End_Medical_Chatbot_Using_LLama2\\data\\Medical_book.pdf',
                           'text': 'Genetic factor\n'
                                   'Some people appear to be more vulnerable '
                                   'to addic-\n'
                                   'tion because their body chemistry '
                                   'increases their sensi-tivity to drugs. '
                                   'Some forms of substance abuse and\n'
                                   'dependence seem to run in families; and '
                                   'this may be the\n'
                                   'result of a genetic predisposition, '
                                   'environmental influ-ences, or a '
                                   'combination of both.\n'
                                 

In [47]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [48]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [49]:
llm=CTransformers(model="D:/OpenAI_Projects/End_to_End_Medical_Chatbot_Using_LLama2/model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [56]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

ValidationError: 1 validation error for RetrievalQA
retriever
  Can't instantiate abstract class BaseRetriever with abstract methods _aget_relevant_documents, _get_relevant_documents (type=type_error)

In [52]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    return_source_documents=True, 
    chain_type_kwargs={}
)

ValidationError: 1 validation error for RetrievalQA
retriever
  Can't instantiate abstract class BaseRetriever with abstract methods _aget_relevant_documents, _get_relevant_documents (type=type_error)